# Skill Challenge - Cab Finder

## Loading the data into memory

Open each file and read the contents. I will be using the json data because it is easier to manipulate without any extra wahala. To do this, I would read the data from the json file as a string, then decode the contents as JSON and then save it to a dictionary using the `json` module.

In [31]:
JSON_DATA_LOC = ".\\Data\\JSON"
LOCATION_LOC = "\\locations.json"
RIDES_LOC = "\\rides.json"
RIDES_SERVICES_LOC = "\\rideservices.json"

Load the data into memory using the `with` keyword

In [32]:
# importing the module
import json

locations = dict({})
rides = dict({})
rideservices = dict({})

# Opening Locations JSON file
with open(JSON_DATA_LOC + LOCATION_LOC) as jf:
    locations_temp = json.load(jf)
    locations = {value["location_id"] : value for value in locations_temp}

# Opening Rides JSON file
with open(JSON_DATA_LOC + RIDES_LOC) as jf:
    rides_temp = json.load(jf)
    rides = {value["ride_id"]: value for value in rides_temp}

# Opening Rides Services JSON file
with open(JSON_DATA_LOC + RIDES_SERVICES_LOC) as jf:
    rideservices_temp = json.load(jf)
    rideservices = {value["rideservice_id"]: value for value in rideservices_temp}


## Task 1

A function that takes in a list of routes, ride services and booked rides then calculate the cost of each ride(price) based on the distance covered and return a list containing the details of each ride and their corresponding prices.

### Calculate distance for a route

Create a function that takes in the latitudes and longitudes of the route's start and destination coordinates

In [33]:
import math

def hervensine_distance(start_cords: 'tuple[float,float]', end_cords: 'tuple[float,float]') -> float:
    earth_radius = 6371.0  # in kilometers
    dlat = math.radians(end_cords[0]-start_cords[0])
    dlon = math.radians(end_cords[1]-start_cords[1])
    a = math.sin(dlat/2)**2 + math.cos(math.radians(start_cords[0])) * \
        math.cos(math.radians(end_cords[0])) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return earth_radius * c

- Creating a store for the results
- Loop throuth the rides, and in each iteration, calculate the distance
- Within the same loop, loop throuth the ridesservices and calculate the price for each rideservice for the distance, then store in the results?

> Because routes to rides is not a 1-1 mapping, I would keep a store of the various prices for eack route in a separate data store. While looping through the rides, if I have calculated the prices forthat route, I'll use that instead.

In [34]:
def get_prices(route, rideservices):
    result = dict({})

    distance = hervensine_distance((route["start_coord_lat"], route["start_coord_long"]), (
            route["destination_coord_lat"], route["destination_coord_long"]))

    for rideservice_id, rideservice in rideservices.items():
        result[rideservice_id] = distance * rideservice["priceperkm"]

    return result


In [35]:
from pprint import pprint


def task1(locations, rideservices, rides):
    result = dict({})
    memo = dict({})

    for ride_id, ride in rides.items():
        route = locations[ride["location_id"]]
        if ride["location_id"] in memo:
            result[ride_id] = {
                "ride_id": ride_id,
                "location_id": ride["location_id"],
                "rideservices_prices": memo[ride["location_id"]]
            }
        else:
            memo[ride["location_id"]] = get_prices(route, rideservices)
            result[ride_id] = {
                "ride_id": ride_id,
                "location_id": ride["location_id"],
                "rideservices_prices": memo[ride["location_id"]]
            }
    return result
        
task1(locations, rideservices, rides)


{1: {'ride_id': 1,
  'location_id': 6,
  'rideservices_prices': {1: 394.3823692966933,
   2: 438.20263255188144,
   3: 306.741842786317,
   4: 368.0902113435804,
   5: 262.9215795311289,
   6: 482.0228958070696}},
 2: {'ride_id': 2,
  'location_id': 4,
  'rideservices_prices': {1: 580.9613968797468,
   2: 645.5126631997186,
   3: 451.85886423980304,
   4: 542.2306370877636,
   5: 387.30759791983115,
   6: 710.0639295196904}},
 3: {'ride_id': 3,
  'location_id': 10,
  'rideservices_prices': {1: 470.9098892702015,
   2: 523.233210300224,
   3: 366.26324721015675,
   4: 439.5158966521881,
   5: 313.93992618013436,
   6: 575.5565313302463}},
 4: {'ride_id': 4,
  'location_id': 9,
  'rideservices_prices': {1: 194.89215328867422,
   2: 216.54683698741582,
   3: 151.58278589119107,
   4: 181.89934306942928,
   5: 129.9281021924495,
   6: 238.2015206861574}},
 5: {'ride_id': 5,
  'location_id': 7,
  'rideservices_prices': {1: 124.1577013118142,
   2: 137.95300145757133,
   3: 96.56710102029993

### Task 2

Leveraging your approach in Task 1, Write a method that takes in a list of routes, ride services, booked rides and computes the best price, the method should return an object containing the name of the ride service and the best price.

In [36]:
def task2(locations, rideservices, rides):
    # Get the output of task 1
    task1_output = task1(locations, rideservices, rides)
    result = dict({})
    memo = dict({})

    for ride_id, data in task1_output.items():
        if data["location_id"] in memo:
            result[ride_id] = {
                "ride_id": ride_id,
                "location_id": data["location_id"],
                "rideservice_name": memo[data["location_id"]]["rideservice_name"],
                "best_price": memo[data["location_id"]]["best_price"]
            }
        else:
            least_value = sorted(data["rideservices_prices"].items(), key=lambda x:x[1])
            memo[data["location_id"]] = {
                "rideservice_name": rideservices[least_value[0][0]],
                "best_price": least_value[0][1]
            }
            result[ride_id] = {
                "ride_id": ride_id,
                "location_id": data["location_id"],
                "rideservice_name": memo[data["location_id"]]["rideservice_name"],
                "best_price": memo[data["location_id"]]["best_price"]
            }
    return result

task2(locations, rideservices, rides)


{1: {'ride_id': 1,
  'location_id': 6,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'priceperkm': 30},
  'best_price': 262.9215795311289},
 2: {'ride_id': 2,
  'location_id': 4,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'priceperkm': 30},
  'best_price': 387.30759791983115},
 3: {'ride_id': 3,
  'location_id': 10,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'priceperkm': 30},
  'best_price': 313.93992618013436},
 4: {'ride_id': 4,
  'location_id': 9,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'priceperkm': 30},
  'best_price': 129.9281021924495},
 5: {'ride_id': 5,
  'location_id': 7,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'priceperkm': 30},
  'best_price': 82.77180087454279},
 6: {'ride_id': 6,
  'location_id': 3,
  'rideservice_name': {'rideservice_id': 5,
   'rideservice_name': 'Gokada',
   'pricepe